In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/SAM-DeepfakeLocalization-local')

In [ ]:
!pip install torchmetrics
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam2.git' # colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
import torch
import numpy as np
import random
import os
import shutil

from parameters import Parameters
from deepfake_datasets import LOCALIZATION_DATASET_PATHS, DETECTION_DATASET_PATHS
from deepfake_datasets.datasets import get_dataloader
from models.sam_localization import get_sam_model
from train.validate import validate_detection, validate_fully_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_f1, compute_mean_ap, compute_mean_acc_detection, compute_mean_ap_detection

In [ ]:
!mkdir -p ../checkpoints/
!wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt

--2025-05-18 21:35:43--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.102, 108.157.254.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184416285 (176M) [application/vnd.snesdev-page-table]
Saving to: ‘../checkpoints/sam2.1_hiera_small.pt’

sam2.1_hiera_small. 100%[===================>] 175.87M   328MB/s    in 0.5s    

2025-05-18 21:35:44 (328 MB/s) - ‘../checkpoints/sam2.1_hiera_small.pt’ saved [184416285/184416285]



In [ ]:
# set seed for reproducibility
SEED = 0
def set_seed():
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

# save image predictions to files
def save_image_localization_scores_to_file(ious, f1_best, f1_fixed, aps, img_paths, params):
    with open(params.save_dir_results + "/scores.txt", 'w') as f:
        f.write(f'image path \t iou \t f1_best \t f1_fixed \t ap\n')
        for iou, f1_b, f1_f, ap, img_path in zip(ious, f1_best, f1_fixed, aps, img_paths):
            f.write(f'{img_path} \t {iou} \t {f1_b} \t {f1_f} \t {ap}\n')

# constants for image processing
MEAN = {
    "imagenet":[0.485, 0.456, 0.406],
    "clip":[0.48145466, 0.4578275, 0.40821073]
}
STD = {
    "imagenet":[0.229, 0.224, 0.225],
    "clip":[0.26862954, 0.26130258, 0.27577711]
}

In [ ]:
# set parameters
params = Parameters()

# set the experiment name and output directory
# alert if the experiment name is not set or the output directory already exists
params.experiment_name = 'test_sam_conv4'
assert params.experiment_name != '', 'Please set the experiment name'
experiment_path = f'/content/experiments/{params.experiment_name}'
if os.path.exists(experiment_path):
    shutil.rmtree(experiment_path)
params.create_output_dirs()

# set data labels to test
params.data_label = 'test'

# uncomment lines below to set new datasets
# new_root_path = '/content/datasets/dolos_data/celebahq/'
# new_dataset_name = 'repaint-p2-9k'
# params.update_dolos_data_paths(new_root_path, new_dataset_name)

# set the model checkpoint path
params.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/SAM-DeepfakeLocalization-local/trained_models/train_new/lama/high_res_feats_0_32_64/best_localization_model_iou_92.7912.pth'
assert params.checkpoint_path != '', 'Please set the checkpoint path'
params.sam_checkpoint_path = '../checkpoints/sam2.1_hiera_small.pt'
params.sam_config_path = 'configs/sam2.1/sam2.1_hiera_s.yaml'

# set model parameters for testing
state_dict = torch.load(params.checkpoint_path, map_location='cpu')
params.decoder_type = state_dict['decoder_type']
params.feature_layer = state_dict['feature_layer']

# set the batch size and num threads
params.batch_size = 32
params.num_threads = 4

In [ ]:
# load the model
model = get_sam_model(params)
model.load_state_dict(state_dict['model'], strict=False)
model = model.cuda()
model.eval()

print('Model loaded successfully!')

Model loaded successfully!


In [ ]:
# prepare the datasets and the results file
if params.task_type == 'fully_supervised_localization':
    dataset_paths = LOCALIZATION_DATASET_PATHS
    with open(os.path.join(params.save_dir_results, 'scores.txt'), 'a') as f:
        f.write('dataset \t iou \t f1_best \t f1_fixed \t ap \n')

In [ ]:
torch.cuda.empty_cache()
# start the testing process
for dataset_path in dataset_paths:
    print(f"Testing on {dataset_path['key']}")
    if 'autosplice' in dataset_path['key']:
        continue
    set_seed()
    os.makedirs(os.path.join(params.save_dir_results, dataset_path['key']), exist_ok=True)

    params.train_dataset = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['key'])
    params.test_fake_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['fake_path'])
    params.test_masks_ground_truth_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['masks_path'])
    if params.task_type == 'detection':
        params.test_real_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['real_path'])

    data_loader = get_dataloader(params)

    # create the directory for dataset results
    os.makedirs(os.path.join(params.save_dir_results, params.train_dataset), exist_ok=True)

    if params.task_type == 'fully_supervised_localization':
        ious, f1_best, f1_fixed, ap, original_img_paths = validate_fully_supervised_localization(
            model, data_loader, dataset_path['key'], params.save_dir_results
        )
        save_image_localization_scores_to_file(ious, f1_best, f1_fixed, ap, original_img_paths, params)

        mean_iou = compute_mean_iou(ious, verbose=True, extra_text=dataset_path['key'])
        mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=dataset_path['key'])
        mean_ap = compute_mean_ap(ap, verbose=True, extra_text=dataset_path['key'])

        with open(os.path.join(params.save_dir_results, 'scores.txt'), 'a') as f:
            f.write(f"{dataset_path['key']} \t {mean_iou:.4f} \t {mean_f1_best:.4f} \t {mean_f1_fixed:.4f} \t {mean_ap:.4f} \n")
    print()

In [ ]:
import shutil
from google.colab import files

filename = os.path.basename(params.checkpoint_path)

parent_dir = os.path.basename(os.path.dirname(params.checkpoint_path))

output_name = f'results_{parent_dir}_{filename}'

shutil.make_archive(output_name, 'zip', f'/content/experiments/test_sam_conv4')
files.download(f'{output_name}.zip')